# DHTML  

페이지가 두 번 깜빡임 - 페이지 리로딩  
=> 서로 다른 형태의 두 개의 돔

dynamic html
요청 - 응답; 뒷단에서 javascript를 이용하여 (AJAX)
일반적으로는 자바스크립트를 통해 받아오는 데이터를 못 가져옴
오늘은 이걸 받아오는 방법을 해볼 것
얘를 이용하여 매크로를 만들 수 있음
내일은 automation에 대해 할 것

이걸 시험해 볼 수 있는 사이트 : example.webscraping.com/places/default/search

In [1]:
import requests
import time

headers = {"user-agent" : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}

# request package를 사용하여 error handling
def download(method, url, param=None, data=None, timeout=1, maxretries=3):
    try:
        resp = requests.request(method, url, params=param, data=data, headers=headers)
        # 한글처럼 아스키가 아닌 문자들을 변환시킬 때 서비스키도 같이 바꿔버림
        resp.raise_for_status()
        #if(resp.headers['Content-Type'].split("/")[0] == "image"):
            #resp.headers['Content-Type'].split("/")[1] #확장자
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and maxretries > 0:
            print(e.response.status_code)
            print(e.response.reason)
            time.sleep(timeout) # 1초 정도 기다림
            print(maxretries)
            print("재시도") #maxretries
            download(method, url, param, data, timeout, maxretries-1)
        else:
            print(e.response.status_code)
            print(e.response.reason)
    return resp

In [21]:
from bs4 import BeautifulSoup
import re
import sqlite3

In [15]:
url = "http://example.webscraping.com/places/ajax/search.json"
param = {
    "search_term" : "korea",
    "page_size" : 10,
    "page" : 0,
}
html = download("get", url, param)
dom = BeautifulSoup(html.text, 'lxml')

In [16]:
requests.compat.urlparse(html.request.url)

ParseResult(scheme='http', netloc='example.webscraping.com', path='/places/ajax/search.json', params='', query='search_term=korea&page_size=10&page=0', fragment='')

In [25]:
dom

<html><body><p>{"records": [{"pretty_link": "</p><div><a href='\"/places/default/view/North-Korea-165\"'><img src='\"/places/static/images/flags/kp.png\"'/> North Korea</a></div>", "country": "North Korea", "id": 4552545}, {"pretty_link": "<div><a href='\"/places/default/view/South-Korea-211\"'><img src='\"/places/static/images/flags/kr.png\"'/> South Korea</a></div>", "country": "South Korea", "id": 4552591}], "num_pages": 1, "error": ""}
</body></html>

In [26]:
html.json()

{'records': [{'pretty_link': '<div><a href="/places/default/view/North-Korea-165"><img src="/places/static/images/flags/kp.png" /> North Korea</a></div>',
   'country': 'North Korea',
   'id': 4552545},
  {'pretty_link': '<div><a href="/places/default/view/South-Korea-211"><img src="/places/static/images/flags/kr.png" /> South Korea</a></div>',
   'country': 'South Korea',
   'id': 4552591}],
 'num_pages': 1,
 'error': ''}

In [38]:
html.json()['records'][0]['pretty_link']

'<div><a href="/places/default/view/North-Korea-165"><img src="/places/static/images/flags/kp.png" /> North Korea</a></div>'

In [37]:
html.json()['records'][1]['pretty_link']

'<div><a href="/places/default/view/South-Korea-211"><img src="/places/static/images/flags/kr.png" /> South Korea</a></div>'

In [39]:
for _ in html.json()["records"]:
    print(_["country"])

North Korea
South Korea


## Selenium
비동기식으로 자바스크립트를 통해 요청을 보내는 애들을 해석할 수 있음
동적으로 변하는 돔을 가져올 수 있도록 도와줌
DHTML을 파싱하거나 수집할 때, 크롤링할 때 얘를 많은 사람들이 이용
자동화가 목적이기 때문에 웹 시스템 개발자 또는 백엔드 개발자들에게 빠르게 피드백을 주기 위함
애자일 방법론과 잘 맞음
rapid feedback to developers
finding defects missed by manual testing
brower automation <- 목적
우리는 다른 방식으로 쓰려고 하는 것
크롬 드라이버 다운로드...
크롬 웹 드라이버라고 검색

In [212]:
from selenium import webdriver

driver = webdriver.Chrome('/Users/eunha/Desktop/2019 빅데이터 청년인재/chromedriver')

In [11]:
# 창 닫고 싶으면 driver.close()해서 닫을 것

In [12]:
%%writefile bigdata.json
{
    "id":"",
    "pw":""
}

Writing bigdata.json


In [13]:
import json
with open("bigdata.json") as fp:
    account = json.load(fp)

In [14]:
account["id"]

'1002galaxy'

In [16]:
driver.page_source

'<html lang="ko"><head>\n<meta charset="utf-8">\n<meta name="Referrer" content="origin">\n<meta http-equiv="Content-Script-Type" content="text/javascript">\n<meta http-equiv="Content-Style-Type" content="text/css">\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta name="viewport" content="width=1100">\n<meta name="apple-mobile-web-app-title" content="NAVER">\n<meta name="robots" content="index,nofollow">\n<meta name="description" content="네이버 메인에서 다양한 정보와 유용한 컨텐츠를 만나 보세요">\n<meta property="og:title" content="네이버">\n<meta property="og:url" content="https://www.naver.com/">\n<meta property="og:image" content="https://s.pstatic.net/static/www/mobile/edit/2016/0705/mobile_212852414260.png">\n<meta property="og:description" content="네이버 메인에서 다양한 정보와 유용한 컨텐츠를 만나 보세요">\n<meta name="twitter:card" content="summary">\n<meta name="twitter:title" content="">\n<meta name="twitter:url" content="https://www.naver.com/">\n<meta name="twitter:image" content="https://s.pstatic.net/static/www

In [17]:
driver.get("http://example.webscraping.com/places/default/search")

In [18]:
inputTag = driver.find_element_by_id("search_term")
inputTag

<selenium.webdriver.remote.webelement.WebElement (session="63bade249a42f5f9f87a542a95ab0824", element="6f23bc08-d4ae-45a1-9a02-1f4d6fe39895")>

In [19]:
inputTag.send_keys("korea") # key를 보냄

In [20]:
driver.find_element_by_xpath("//input[@type='submit']").click()

In [22]:
dom = BeautifulSoup(driver.page_source, "lxml")
dom.select("#results")

[<div id="results"><table><tbody><tr><td><div><a href="/places/default/view/North-Korea-165"><img src="/places/static/images/flags/kp.png"/> North Korea</a></div></td><td><div><a href="/places/default/view/South-Korea-211"><img src="/places/static/images/flags/kr.png"/> South Korea</a></div></td></tr></tbody></table></div>]

결과 잘 들어있음!
*__반영된 돔을 가져올 수 있다!__*
지금까지 했던 request, response와는 조금 다르다
tag를 찾아서 무언가 탐색해야 하는 거면 셀레니움만으로 안 되고 bs의 힘을 빌려야 함

In [23]:
driver.find_element_by_css_selector("#results").text

'North Korea\nSouth Korea'

In [24]:
driver.get("https://www.naver.com")

In [26]:
driver.find_element_by_class_name("ico_local_login.lang_ko").click()

In [30]:
inputid.clear()
inputid = driver.find_element_by_id("id")
inputid.send_keys(account["id"])

In [32]:
inputpw.clear()
inputpw = driver.find_element_by_id("pw")
inputpw.send_keys("1002dmskzla12")

In [35]:
driver.get("https://mail.naver.com")

In [48]:
for _ in driver.find_elements_by_css_selector("strong.mail_title"):
    print('[CSS]', _.text)

[CSS] (광고) [원더브라] 단 하루 초특가전! 브라팬티 종 18,900원★ 신한카드 5% 즉시할인까지!
[CSS] (광고) 이런 상품 어때요? 동서가구 잠자리 천연라텍스 바디필로우 DF910283 외
[CSS] (광고) 오늘의 최신 토익RC문제 무료학습 ▶
[CSS] (광고) 7월의 엑셀메일 _ 총무관리 BEST 서식 확인하고가세요~!
[CSS] 새로운 환경에서 로그인 되었습니다.
[CSS] (광고) [10분어택] 천도복숭아 2kg 2,900원
[CSS] (광고) 7월에 갈만한 곳, 29CM 컬처캘린더
[CSS] (광고) 초기비용 0원! 국내최저가 장기렌트 지금확인하세요~
[CSS] Your Steam account: Access from new web or mobile device
[CSS] (광고) 7월 감사선물 4종 쿠폰팩 + 인생날 D-1!
[CSS] (광고) 배송비 없는 뷰티페어 시작 + 카드사 즉시할인
[CSS] (광고) 쉬즈미스/리스트 원피스 외 몽땅 8,900원
[CSS] EUNHA 님, 오사카 막바지 특가를 확인하세요!
[CSS] [성균웹진 423호] 교환학생제도 알아보기
[CSS] (광고) 10% 할인쿠폰이 도착했습니다.
[CSS] (광고) 생활/주방용품 총집합! 구매시 10% 적립금 증정!
[CSS] [커먼유니크] galaxy1002 고객님 상품이 발송되었습니다.
[CSS] [커먼유니크] galaxy1002 고객님 상품이 발송되었습니다.
[CSS] (광고) 떠나자! 바캉스 최대 60%
[CSS] (광고) [티몬데이 앵콜] 베스트 상품만 엄선했습니다
[CSS] (광고) 종근당건강 GS단독 BIG EVENT! 특가+쿠폰+적립금+사은품!
[CSS] (광고) 오늘의 최신 토익RC문제 무료학습 ▶
[CSS] (광고) [10분어택] 닭가슴살 소시지 10팩 4,900원!
[CSS] (광고) ‘어쨌든 모험여행’ 3박 4일 캠핑카 로드 트립 참가자를 모집합니다.
[CSS] (광고) ★특가★ [무배특가] (2+1) 1회용 행주 8,900원 / [쉿!비밀

In [51]:
for _ in driver.find_elements_by_xpath("//div[@class='mTitle']//span/strong"):
    print('[XPATH]', _.text)

[XPATH] (광고) [원더브라] 단 하루 초특가전! 브라팬티 종 18,900원★ 신한카드 5% 즉시할인까지!
[XPATH] (광고) 이런 상품 어때요? 동서가구 잠자리 천연라텍스 바디필로우 DF910283 외
[XPATH] (광고) 오늘의 최신 토익RC문제 무료학습 ▶
[XPATH] (광고) 7월의 엑셀메일 _ 총무관리 BEST 서식 확인하고가세요~!
[XPATH] 새로운 환경에서 로그인 되었습니다.
[XPATH] (광고) [10분어택] 천도복숭아 2kg 2,900원
[XPATH] (광고) 7월에 갈만한 곳, 29CM 컬처캘린더
[XPATH] (광고) 초기비용 0원! 국내최저가 장기렌트 지금확인하세요~
[XPATH] Your Steam account: Access from new web or mobile device
[XPATH] (광고) 7월 감사선물 4종 쿠폰팩 + 인생날 D-1!
[XPATH] (광고) 배송비 없는 뷰티페어 시작 + 카드사 즉시할인
[XPATH] (광고) 쉬즈미스/리스트 원피스 외 몽땅 8,900원
[XPATH] EUNHA 님, 오사카 막바지 특가를 확인하세요!
[XPATH] [성균웹진 423호] 교환학생제도 알아보기
[XPATH] (광고) 10% 할인쿠폰이 도착했습니다.
[XPATH] (광고) 생활/주방용품 총집합! 구매시 10% 적립금 증정!
[XPATH] [커먼유니크] galaxy1002 고객님 상품이 발송되었습니다.
[XPATH] [커먼유니크] galaxy1002 고객님 상품이 발송되었습니다.
[XPATH] (광고) 떠나자! 바캉스 최대 60%
[XPATH] (광고) [티몬데이 앵콜] 베스트 상품만 엄선했습니다
[XPATH] (광고) 종근당건강 GS단독 BIG EVENT! 특가+쿠폰+적립금+사은품!
[XPATH] (광고) 오늘의 최신 토익RC문제 무료학습 ▶
[XPATH] (광고) [10분어택] 닭가슴살 소시지 10팩 4,900원!
[XPATH] (광고) ‘어쨌든 모험여행’ 3박 4일 캠핑카 로드 트립 참가자를 모집합니다.
[X

In [42]:
driver.page_source

'<html lang="ko"><head>\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge">\n\t<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\n\t\n\t\t\n\t\t\n\t\t\t\n\t\t\t\t\n\t\t\t\t\t<link rel="stylesheet" type="text/css" href="/css/mail2_le.min.190611.css">\n\t\t\t\t\t<link rel="stylesheet" type="text/css" href="/css/mail_skin.min.190611.css">\n\t\t\t\t\n\t\t\t\t\n\t\t\t\n\t\t\n\t\n\n\t\n\t\t\n\t\t\t\n\t\t\t\t\n\t\t\t\t\n\t\t\t\t\t<link rel="dns-prefetch" href="//ssl.pstatic.net">\n\t\t\t\t\t<link rel="dns-prefetch" href="//lcs.naver.com">\n\t\t\t\t\t<link rel="dns-prefetch" href="//cc.naver.com">\n\t\t\t\t\t<link rel="dns-prefetch" href="//static.nid.naver.com">\n\t\t\t\t\t<link rel="dns-prefetch" href="//phinf.pstatic.net">\n\t\t\t\t\t<link rel="stylesheet" type="text/css" href="/css/mail_me.min.190611.css">\n\t\t\t\t\n\t\t\t\n\t\t\t<link rel="shortcut icon" href="/favicon.ico?2">\n\t\t\t\n\t\t\n\n\t\t\n\t\n\n\t\n\t\t\n\t\n\t<title>보낸메일함 : 네이버 메일</title>\n\t<script typ

In [53]:
inputid = driver.find_element_by_xpath("//input[@name='userid']")
inputid.send_keys("1002galaxy")

In [54]:
inputpw = driver.find_element_by_xpath("//input[@name='password']")
inputpw.send_keys("dmskzla12")

In [55]:
driver.find_element_by_xpath("//input[@type='submit']").click()

In [58]:
driver.find_element_by_css_selector(".item_wrap.bu6").click()

In [63]:
for cookie in driver.get_cookies():
#     print(cookie["name"], cookie["value"])
    session.cookies.set(cookie["name"], cookie["value"])

In [61]:
import requests

session = requests.Session()

In [64]:
session.cookies

<RequestsCookieJar[Cookie(version=0, name='NID_AUT', value='M6o3e3naGtcyf1BPT87MYp2T2EdAlQ1GL+uApJwq7vNGHMB9PEgUmtNktFIsx5Bn', port=None, port_specified=False, domain='', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False), Cookie(version=0, name='NID_JKL', value='zgkaNdEEOJCSJH8m5T+pP9lGkCAONgTym46qJ4KQboE=', port=None, port_specified=False, domain='', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False), Cookie(version=0, name='NID_SES', value='AAABehE6NxAplRIccd9/4aSyhESqFyIzySb6/IIOpXJYmJIqVAFurw4X2jBdoniklf8GcxY33lHLNJdPLntxB/sjMUxKMQrjM9V6f8bs8p/ER7BJQMGBg3wQQaxVh5M4PqyECu3+I2Y+6weXUjbJrtIHbSFwMkdlOIXvV5aBXDl+mxZLGxU/j9D59pMyczDof28z5fKsAogx3GyZIqoizd0Rvl++e9yaebf6zAv22bLaspNkFFsc+aBBKs6sjs6IGv4kucpe

In [65]:
html = session.get("https://mail.naver.com/#%7B%22fClass%22%3A%22list%22%2C%22oParameter%22%3A%7B%22page%22%3A1%2C%22sortField%22%3A1%2C%22sortType%22%3A0%2C%22folderSN%22%3A%225%22%2C%22type%22%3A%22%22%2C%22isUnread%22%3Afalse%7D%7D")
html.text

'\n\n\n<!DOCTYPE HTML>\n\n\t\n\t\n\t\n\t\n\t\n\t\t<html lang="ko">\n\t\n\n\n<head>\n\t<meta http-equiv="X-UA-Compatible" content="IE=edge">\n\t<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\n\t\n\t\t\n\t\t\n\t\t\t\n\t\t\t\t\n\t\t\t\t\t<link rel="stylesheet" type="text/css" href="/css/mail2_le.min.190611.css">\n\t\t\t\t\t<link rel="stylesheet" type="text/css" href="/css/mail_skin.min.190611.css">\n\t\t\t\t\n\t\t\t\t\n\t\t\t\n\t\t\n\t\n\n\t\n\t\t\n\t\t\t\n\t\t\t\t\n\t\t\t\t\n\t\t\t\t\t<link rel="dns-prefetch" href="//ssl.pstatic.net" />\n\t\t\t\t\t<link rel="dns-prefetch" href="//lcs.naver.com" />\n\t\t\t\t\t<link rel="dns-prefetch" href="//cc.naver.com" />\n\t\t\t\t\t<link rel="dns-prefetch" href="//static.nid.naver.com" />\n\t\t\t\t\t<link rel="dns-prefetch" href="//phinf.pstatic.net" />\n\t\t\t\t\t<link rel="stylesheet" type="text/css" href="/css/mail_me.min.190611.css">\n\t\t\t\t\n\t\t\t\n\t\t\t<link rel="shortcut icon" href="/favicon.ico?2">\n\t\t\t\n\t\t\n\n\t\

In [66]:
html = session.post("https://mail.naver.com/json/list/?page=1&sortField=1&sortType=0&folderSN=0&type=&isUnread=false&u=1002galaxy")
html.json()

{'currentTime': 1563337576,
 'lastPage': 312,
 'Message': '',
 'pageSize': 30,
 'unreadCount': 8896,
 'mailData': [{'mailSN': 21349,
   'folderSN': 0,
   'folderName': None,
   'iconType': 0,
   'status': 2097152,
   'from': {'name': 'GS SHOP', 'email': 'customer@gsshop.com'},
   'subject': '(광고) [원더브라] 단 하루 초특가전! 브라팬티 종 18,900원★ 신한카드 5% 즉시할인까지! ',
   'receivedTime': 1563330162,
   'sentTime': 1563330162,
   'size': 12094,
   'attachCount': 0,
   'iDomainEmail': '1002galaxy@naver.com',
   'priority': 3,
   'securityLevel': None,
   'reSend': False,
   'color': '#ffffff',
   'preview': '',
   'mimeSN': '1563330162.506137.48300.50944',
   'toList': [{'name': '김은하', 'email': '1002galaxy@naver.com'}],
   'spamType': None,
   'attachInfo': [],
   'threadId': '<201907171122422453@smtp56.gsshop.com_Ems@smtp56.gsshop.com>',
   'bccList': [],
   'receiveByBcc': False,
   'threadCount': 0,
   'unreadThreadCount': 0,
   'firstLocatedTime': 1563330162,
   'remindAt': 0,
   'threadRemindAt': 0,
   

In [69]:
from html import unescape
for _ in html.json()["mailData"]:
    print(unescape(_["subject"]))

(광고) [원더브라] 단 하루 초특가전! 브라팬티 종 18,900원★ 신한카드 5% 즉시할인까지! 
(광고) 이런 상품 어때요? 동서가구 잠자리 천연라텍스 바디필로우 DF910283 외
(광고) 오늘의 최신 토익RC문제 무료학습 ▶
(광고) 7월의 엑셀메일 _ 총무관리 BEST 서식 확인하고가세요~!
새로운 환경에서 로그인 되었습니다.
(광고) [10분어택] 천도복숭아 2kg 2,900원 
(광고) 7월에 갈만한 곳, 29CM 컬처캘린더
(광고) 초기비용 0원! 국내최저가 장기렌트 지금확인하세요~
Your Steam account: Access from new web or mobile device
(광고) 7월 감사선물 4종 쿠폰팩 + 인생날 D-1!
(광고) 배송비 없는 뷰티페어 시작 + 카드사 즉시할인
(광고) 쉬즈미스/리스트 원피스 외 몽땅 8,900원 				
EUNHA 님, 오사카 막바지 특가를 확인하세요!
[성균웹진 423호] 교환학생제도 알아보기
(광고) 10% 할인쿠폰이 도착했습니다.
(광고) 생활/주방용품 총집합! 구매시 10% 적립금 증정!
[커먼유니크] galaxy1002 고객님 상품이 발송되었습니다.
[커먼유니크] galaxy1002 고객님 상품이 발송되었습니다.
(광고) 떠나자! 바캉스 최대 60%
(광고) [티몬데이 앵콜] 베스트 상품만 엄선했습니다
(광고) 종근당건강 GS단독 BIG EVENT! 특가+쿠폰+적립금+사은품!
(광고) 오늘의 최신 토익RC문제 무료학습 ▶
(광고) [10분어택] 닭가슴살 소시지 10팩 4,900원!
(광고) ‘어쨌든 모험여행’ 3박 4일 캠핑카 로드 트립 참가자를 모집합니다.
(광고) ★특가★ [무배특가] (2+1) 1회용 행주 8,900원 / [쉿!비밀특가] 위닉스 제습기 / [정품] 프라다 버킷백 (2컬러) 
와이드모바일 이용약관 개정 안내
(광고) 디커버스 반팔 티 3장 9,900원 + 무료배송				
(광고) 이런 상품 어때요? 제이앤제이 스마트 가꾸로 골프 장우산 외
(광고) 가구/인테리어 최대 

In [70]:
html = requests.post("https://mail.naver.com/json/list/?page=1&sortField=1&sortType=0&folderSN=0&type=&isUnread=false&u=1002galaxy")
html.text
# 쿠키 값이 없으니까 fail <- loginstatur : nologin

'{"Result":"FAIL","Message":"", "LoginStatus":"NOLOGIN"}'

In [75]:
html = session.post("https://mail.naver.com/json/list/?page=1&sortField=1&sortType=0&folderSN=5&type=&isUnread=false&u=1002galaxy")
html.json()

{'currentTime': 1563337923,
 'lastPage': 1,
 'Message': '',
 'pageSize': 30,
 'unreadCount': 2,
 'mailData': [{'mailSN': 21341,
   'folderSN': 5,
   'folderName': None,
   'iconType': 0,
   'status': 2097152,
   'from': {'name': '파이널판타지14', 'email': 'FFXIV@actoz.com'},
   'subject': '(광고)[파이널판타지14]엔 여름 맞이 풍성한 이벤트가 가득! 지금 바로 접속하세요!',
   'receivedTime': 1563275536,
   'sentTime': 1563275394,
   'size': 5558,
   'attachCount': 0,
   'iDomainEmail': '1002galaxy@naver.com',
   'priority': 3,
   'securityLevel': None,
   'reSend': False,
   'color': '#ffffff',
   'preview': '',
   'mimeSN': '1563275536.776819.46866.22272',
   'toList': [{'name': '1002galaxy 님', 'email': '1002galaxy@naver.com'}],
   'spamType': 'SYSTEM_DEFINE',
   'attachInfo': [],
   'threadId': '',
   'bccList': [],
   'receiveByBcc': False,
   'threadCount': 0,
   'unreadThreadCount': 0,
   'firstLocatedTime': 1563275536,
   'remindAt': 0,
   'threadRemindAt': 0,
   'threadRemindFlag': 0,
   'category': -1,
   'lastLocated

In [82]:
html = session.get("https://section.cafe.naver.com/cafe-home-api/v1/homepc?myCafeCount=10&articleCount=3&useMyCafeEvent=true")
for _ in html.json()['message']['result']['myCafe']['cafes']:
    print(_['cafeName'])

Marauder's Map
낙대부고 오케스트라부
스타듀밸리 한국 커뮤니티
쇼핑1번지 보부상 [ 신상/중고나라/패션/가전/게임/버스폰/원룸/
여름 후유증
NAVER TRPG CAFE
성대문학편집부
혼자 공부하는 C언어
권규호 수능국어 연구실
고전문학반


In [85]:
for _ in driver.find_elements_by_xpath("//div[@class='user_mycafe_list']//div[@class='user_mycafe_info']/a/strong[@class='name']"):
    print(_.text)

Marauder's Map
낙대부고 오케스트라부
스타듀밸리 한국 커뮤니티
쇼핑1번지 보부상 [ 신상/중고나라/패션/가전/게임/버스폰/원룸/
여름 후유증
NAVER TRPG CAFE
성대문학편집부
혼자 공부하는 C언어
권규호 수능국어 연구실
고전문학반


In [91]:
for _ in driver.find_elements_by_css_selector(".user_mycafe_list .user_mycafe_info a strong.name"):
    print(_.text)

Marauder's Map
낙대부고 오케스트라부
스타듀밸리 한국 커뮤니티
쇼핑1번지 보부상 [ 신상/중고나라/패션/가전/게임/버스폰/원룸/
여름 후유증
NAVER TRPG CAFE
성대문학편집부
혼자 공부하는 C언어
권규호 수능국어 연구실
고전문학반


In [92]:
driver.get("http://www.cine21.com/db/mag/news_section/?section=005003032")

In [101]:
for _ in driver.find_elements_by_css_selector(".sec_news a span.tit"):
    print(_.text)

[김혜리의 영화의 일기] ‘디즈니’랜드
[김혜리의 영화의 일기] 유효기간
[김혜리의 영화의 일기] 재활용
[김혜리의 영화의 일기] 탄식
[김혜리의 영화의 일기] 시즌 피날레
[김혜리의 영화의 일기] 사중주
[김혜리의 영화의 일기] 신과 함께


In [104]:
driver.find_element_by_css_selector("#gnb_name2 + a.gnb_btn_login").click()
# +:형제
# 이건 오류가 나야 정상; ElementNotInteractableException
# 코드에 없다는 얘기가 아님
# 페이지가 로딩이 다 안 되면 이 에러가 뜸
# 그냥 일반 페이지들도

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=75.0.3770.142)


In [105]:
dom = BeautifulSoup(driver.page_source, "lxml")
dom.select("#gnb_name2 + a.gnb_btn_login")
# dom에선 보임

[<a class="gnb_btn_login" href="https://nid.naver.com/nidlogin.logout?returl=https://section.cafe.naver.com/Logout.nhn?alreadyNaverLogout=true" id="gnb_logout_button"><span class="gnb_bg"></span><span class="gnb_bdr"></span><span class="gnb_txt">로그아웃</span></a>]

In [170]:
driver.get(baseURL)

In [177]:
dom = BeautifulSoup(driver.page_source, "lxml")
thisurllink=[]
for _ in driver.find_elements_by_css_selector("div.main_component.droppable ul li a"):
    if len(_.get_attribute("href")) < 2 or _.get_attribute("href").startswith("https://news.naver.com/main/read.nhn")==False:
        continue
    if _.get_attribute("href").startswith("https"):
        thisurllink.append(_.get_attribute("href"))
    else:
        thisurllink.append(requests.compat.urljoin("https://news.naver.com/", _.get_attribute("href")))
thisurllink

['https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=001&aid=0010960167',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=001&aid=0010959769',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=081&aid=0003014746',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=021&aid=0002397235',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=047&aid=0002233813',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=001&aid=0010960166',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=079&aid=0003249907',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=448&aid=0000278107',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=437&aid=0000215223',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=008&aid=0004249914',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=101&oid=277&aid=0004503149',
 'https://

In [206]:
sthisurllink=[]
for _ in driver.find_elements_by_css_selector("div.main_component.droppable ul li a"):
    sthisurllink.append(_)
sthisurllink

[<selenium.webdriver.remote.webelement.WebElement (session="7e520bce6c2ce67a8a85bae78e31386b", element="68977b35-9a21-4280-972d-d6916a274822")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7e520bce6c2ce67a8a85bae78e31386b", element="7f58f142-0c60-4dda-890a-bca1f0d77951")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7e520bce6c2ce67a8a85bae78e31386b", element="c7a71a20-1fdc-4aa2-b1b4-18fec7222177")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7e520bce6c2ce67a8a85bae78e31386b", element="984f5952-2154-4723-bbe8-65124f73d467")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7e520bce6c2ce67a8a85bae78e31386b", element="c8b6d45c-7906-4b60-9c55-0dc6fa180ec6")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7e520bce6c2ce67a8a85bae78e31386b", element="b4073f07-098b-4c68-b23d-311505a19719")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7e520bce6c2ce67a8a85bae78e31386b", element="ff76623f-457b-47b9-a90e-c6

In [210]:
sthisurllink[0]

In [205]:
driver.get(baseURL)

selenium.webdriver.remote.webelement.WebElement (session="7e520bce6c2ce67a8a85bae78e31386b", element="755b120d-c9c7-49c7-b7a3-04d156b25f77")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7e520bce6c2ce67a8a85bae78e31386b", element="fae5e544-9779-4e76-8227-a10db18cf2a2")>,

In [204]:
sthisurllink[1].send_keys(Keys.ENTER)

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=75.0.3770.142)


In [197]:
from selenium.webdriver.common.keys import Keys

In [180]:
import time
baseURL = "https://news.naver.com/"
contents = []
for href in thisurllink:
    driver.get(href)
    contents.append((driver.find_element_by_css_selector("h3#articleTitle").text, driver.find_element_by_css_selector("div#articleBodyContents").text))

In [179]:
for _ in contents:
    print(_[0])

'잠망경 추정 물체 목격' 행담도휴게소 방문객들 "설마…"
삼성바이오 대표, 자사주 매입비용 돌려받아 30억 횡령 정황
“김준기, 신사로 변하더니 또 성폭행…짐승처럼 보였다”
정두언 빈소, 정치인 조문 이어져… 부검은 안하기로
나경원 "출석놀이로 야당 겁박"...이인영 "당당히 조사받아라"
국정원 "금강산관광 재개는 유엔제재상 '벌크캐시' 문제와 연결"
[영상]'단식농성' 지나친 손학규 "새 혁신위원장 못 구해"
잠망경 추정물 수색 종료…軍 "어망 부표로 추정"
청 "일 '제3국 중재위서 징용배상 판결 논의' 수용 불가"
정두언 빈소 찾은 '옛 동지들'…"이루 말할 수 없는 슬픔"
3기 신도시 교통 대책이 스마티 모빌리티?
[고졸이라서 상] 예비군 2박3일, 자격증도 바로 못 따…40%는 대학으로
똘똘한 비상장 계열사 SK(주) 몸값 높이나
늑장에 깜깜이 논란…주먹구구식 증액으로 치닫는 추경 예산안
복합제로 무장한 국산 신약 매출 본격 창출
"서울대 왜소해지니 점거 풀라"는 교수들에 학생들 반발
국세청, 강남·홍대 유명 클럽 ‘탈탈’ 턴다…수십억대 추징 예고
용역업체 직원 지게차로 치어 숨지게 한 50대 '집유'
“불륜녀 ○○○” 1인 시위한 60대… 허위사실 명예훼손으로 재판
조희연 교육감 "자사고·외고 폐지 국민 공론화 하자"
하루 450㎏ 먹는 코끼리, 기후변화 줄이는 ‘착한 식성’
[단독]전국 27개 수소충전소 안전 점검 받는다..“고장 방지 차원”
[엄마의 이름을 찾아서] 남편과 나의 심리검사 결과가 알려준 충격적 사실
화장실서 볼일 본 후 손 안 씻으면 '간염' 위험
한·일 기독교계 "일본, 분단상황 이용·조장 시도 중단하라"
높이 치솟은 가발 벗겨보니...스페인서 코카인 500g 밀반입 적발
미중 무역협상, 재개 못하고 좌초되나…트럼프, “갈 길 멀다. 시진핑과 예전만큼 가깝지 않아”
일본 "한국, 중재위 응할 의무 있어... 강력히 요구할 것"
워터파크 슬라이드에서 ‘쿵’…하반신 마비 위기 처한 英 남성
‘김정은 마이바흐’ 네→중→일→한→러 거쳐

In [211]:
driver.close()

In [213]:
# 렌더링 때문에 오류
#driver.implicitly_wait(sec) 
# 몇 sec만큼 기다릴 건지 <- 특정 element를 찾을 때까지 무조건 이만큼 기다림
# sleep으로 대체해도 됨 위에꺼는 얘는 안 쓸 거임

# 우리가 쓸 것은 아래!
# 해당 이벤트가 무시되거나 나타날 때까지 계속 검사하면서 최대 (정의해둔) 초만큼 기다림

In [214]:
driver.find_element_by_css_selector("#gnb_name2 + a.gnb_btn_login").click()

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=75.0.3770.142)


지금 화면에서 안 보여서 나는 오류

이 exeption이 더이상 안 나타날 때까지 기다려주면 된다!

In [215]:
from selenium.webdriver.support.ui import WebDriverWait

In [216]:
from selenium.common.exceptions import ElementNotVisibleException
wait = WebDriverWait(driver, 10, 0.5,[ElementNotVisibleException])

In [218]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [222]:
wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#gnb_name2 + a.gnb_btn_login"))
driver.find_element_by_css_selector("#gnb_name2 + a.gnb_btn_login").click()

In [223]:
wait.until(lambda x:x.find_element_by_css_selector("#gnb_name2 + a.gnb_btn_login"))
driver.find_element_by_css_selector("#gnb_name2 + a.gnb_btn_login")

<selenium.webdriver.remote.webelement.WebElement (session="a33534fa4a065a79a86da4d8802a8f27", element="18846d5b-4348-4422-a557-7d66c6d41513")>

In [224]:
logout = driver.find_element_by_css_selector("#gnb_name2 + a.gnb_btn_login")

In [225]:
logout.is_displayed()

False

In [228]:
iframe = driver.find_element_by_name("da_iframe_time")
driver.switch_to_frame(iframe)

/Users/eunha/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: use driver.switch_to.frame instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [229]:
driver.find_element_by_tag_name("img").get_attribute("src")

'https://ssl.pstatic.net/tveta/libs/1242/1242461/931c8cbd4141fa0965c1_20190715175329354.jpg'

In [230]:
driver.switch_to_default_content()

/Users/eunha/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.default_content instead
  """Entry point for launching an IPython kernel.


In [232]:
driver.window_handles

['CDwindow-484087356FED6A810A140202E0AF9CB3']

## driver.switch_to_window()

In [233]:
driver.get('https://www.kt.com')

In [234]:
driver.find_element_by_css_selector("span.quick > a").click()

In [235]:
driver.switch_to_window(driver.window_handles[-1])

/Users/eunha/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.window instead
  """Entry point for launching an IPython kernel.


In [236]:
driver.find_element_by_id("userId").send_keys("test")

In [237]:
driver.close()

In [238]:
driver.switch_to_window(driver.window_handles[0])

/Users/eunha/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.window instead
  """Entry point for launching an IPython kernel.


In [239]:
from selenium.webdriver.common.keys import Keys

In [241]:
Keys.CONTROL+"t" # new tap 새탭

'\ue009t'

In [242]:
Keys.CONTROL+"n" #new window 새창

'\ue009n'

In [247]:
driver.find_element_by_tag_name("body").send_keys(Keys.CONTROL+Keys.TAB)

In [248]:
driver.execute_script("window.open();")

In [249]:
driver.execute_script("javascript:window.open('about:blank');")

In [276]:
driver.switch_to_window(driver.window_handles[0])

/Users/eunha/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.window instead
  """Entry point for launching an IPython kernel.


In [251]:
driver.switch_to_window(driver.window_handles[1])

/Users/eunha/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.window instead
  """Entry point for launching an IPython kernel.


In [277]:
driver.close()

In [260]:
driver.get(baseURL)

In [272]:
newwincontents=[]
for _ in driver.find_elements_by_css_selector("div.main_component.droppable ul li a"):
    if len(_.get_attribute("href")) < 2 or _.get_attribute("href").startswith("https://news.naver.com/main/read.nhn")==False:
        continue
    if _.get_attribute("href").startswith("https"):
        href = _.get_attribute("href")
#         driver.execute_script("javascript:console.log('"+ title +')")
        driver.execute_script("javascript:window.open('about:blank');")
        driver.switch_to_window(driver.window_handles[1])
        driver.get(href)
        print(driver.find_element_by_css_selector("h3#articleTitle").text)
        driver.close()
        driver.switch_to_window(driver.window_handles[0])
    else:
        href = requests.compat.urljoin("https://news.naver.com/", _.get_attribute("href"))
#         driver.execute_script("javascript:console.log('"+ title +"')")
        driver.execute_script("javascript:window.open('about:blank');")
        driver.switch_to_window(driver.window_handles[1])
        driver.get(href)
        print(driver.find_element_by_css_selector("h3#articleTitle").text)
        driver.close()
        driver.switch_to_window(driver.window_handles[0])

/Users/eunha/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: use driver.switch_to.window instead
  


분당수순 평화당, 내홍 지속 바른미래당


/Users/eunha/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: use driver.switch_to.window instead
  if sys.path[0] == '':


합참 “어망부표로 추정·구조상 잠수함정 침투 어려워”…‘잠망경 소동’ 종료
'성범죄' 김준기, 귀국 불투명…범죄인인도 청구할듯(종합)
8개월간 이어진 '삼성바이오 분식회계' 수사 막바지
"韓日 관계 개선 희망...美도 돕겠다"
법사위 파행…한국-바른미래 "정경두 해임안 처리 합의돼야"
분당 수순 평화당 "제3지대 신당" vs "분열선동 중단하라"
[전문]정청래 "정두언 이렇게 사무치게 그리울 줄은···" 재차 심경 고백
강원지역 군부대 해체·이전…주민들 "대책 세워달라" 반발
與 "추경 이미 늦어" vs 野 "4조7000억 삭감"
전선업계도 '발등의 불'···명노현 LS전선 사장 일본行
상한제 시작도 전에 수도권 분양물량 `뚝`
유니클로, 사과했지만 '진정성 있나'…정작 홈페이지엔 공지문 없어(종합)
1년간 5만건 팔린 달러보험..."손실 우려" 경고한 금감원
"상한제 전에"...과천주공1 후분양도 '빨리'
“후불로 준다더니”…초보 번역가들 등친 50대 구속
1원 보내고 이름 칸에 ‘백만 원’…모바일뱅킹 사기
현대중 노조, 올해 임금협상 파업 투표 가결
"김준기 전 회장, 성폭행 전후 짐승↔신사 왔다갔다" 前가사도우미 주장
'동물국회' 수사서 검경 갈등 재연될까
제주 전역에 장맛비…태풍 다나스 제주 향해 북상(종합)
"칼로리 1/3, 아이스크림 맘껏 즐기세요"…미국 뒤흔든 '헤일로탑' 국내 상륙
현대·기아차, 코나·니로EV 전기차 유럽서 '씽씽'…상반기 2만대 돌파
각선미 위해 종아리 보톡스… 못 걷고 혈액순환 안 될 수도
[연구] 여성, 남성보다 알츠하이머병 발병률 높다
日 언론 "文 정권 일본 비판은 착각"
中, '어머니 복수' 이웃 일가족 3명 살해 남성 사형 집행
아베에 "물러가!" 외친뒤 끌려간 남성…日 강타한 논란 영상
"부끄럽다" 獨 트럼프 조부 고향서도 인종차별 발언에 '한숨'
"일본 기업 압류 자산 매각 땐 모든 선택지 놓고 대응"
美 포브스 "LG 듀얼 스크린, 현실적인 폴더블 옵션"
앞뒤로 택시 손 잡아준 국토부
[단독]장정욱 AWS코리아 

In [273]:
driver.execute_script("javascript:console.log('eunha')")